HW4
1h

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


df = pd.read_csv('Final_Imputed_Whole.csv',index_col=0)
df_train = df.iloc[0:1495,:]
df_test = df.iloc[1495:1994,:]
df_test.reset_index(drop=True,inplace=True)
X = df_train.drop(columns=['ViolentCrimesPerPop'])
X_final_test = df_test.drop(columns=['ViolentCrimesPerPop'])
y = df_train['ViolentCrimesPerPop']
y_final_test = df_test['ViolentCrimesPerPop']
print(X)
print(y)
print(X_final_test)
print(y_final_test)

      population  householdsize  racepctblack  racePctWhite  racePctAsian  \
0           0.19           0.33          0.02          0.90          0.12   
1           0.00           0.16          0.12          0.74          0.45   
2           0.00           0.42          0.49          0.56          0.17   
3           0.04           0.77          1.00          0.08          0.12   
4           0.01           0.55          0.02          0.95          0.09   
...          ...            ...           ...           ...           ...   
1490        0.00           0.59          0.01          0.69          1.00   
1491        0.16           0.62          0.03          0.74          0.48   
1492        0.01           0.36          0.95          0.24          0.03   
1493        0.06           0.53          0.01          0.98          0.05   
1494        0.05           0.43          0.08          0.65          0.12   

      racePctHisp  agePct12t21  agePct12t29  agePct16t24  agePct65up  ...  

The code here is exactly the same as that in g, except Lasso has been used
We still use 5 fold CV to pick the best alpha/lambda
The range of alpha is kept the same
(Some experimentation with this range led to the conclusion that alpha, indeed, was minimized for an alpha within this range)
(Thus we didn't need to check over a wider range, as the result always corresponded with alpha being 0.005 as seen in the result)

In [16]:
alpha = np.array([5*(10**(-3)),5*(10**(-2)),5*(10**(-1)),5*(10**(0)),5*(10**(1))])
alpha_pointer = np.arange(0,5)
alpha_mse_sum = np.zeros(5)
kf = KFold(n_splits=5,shuffle=True)
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    i=0
    for i in alpha_pointer:
        lasso_reg = linear_model.Lasso(alpha=alpha[i],fit_intercept=True,normalize=False) #Already Normalized, Fit Intercept
        lasso_fit = lasso_reg.fit(X_train,y_train)
        y_predicted = lasso_fit.predict(X_test)
        mse_interim = mean_squared_error(y_test,y_predicted)
        alpha_mse_sum[i] = alpha_mse_sum[i] + mse_interim #alpha_mse_sum stores mse in indices corresponding to alphas, and 
                                                          #it keeps summing mse's for each alpha over every single fold, 
                                                          #final mse obtained by dividing by 5 
                
final_alpha_mse = alpha_mse_sum/5 #alpha mse were summed, dividing by 5 gives us the average mse for each alpha over 5-folds


selected_alpha_index = np.argmin(final_alpha_mse) #returns index with minimal mse, corresponding to index for alpha

print('The alpha selected, via 5-fold CV is : \n',alpha[selected_alpha_index])

final_lasso = linear_model.Lasso(alpha=alpha[selected_alpha_index],fit_intercept=True,normalize=False)

final_fit = final_lasso.fit(X,y) #fitting on the entire training set

y_final_predicted = final_fit.predict(X_final_test)

print('The final y predicted over test set with the best selected alpha/lambda is : \n',y_final_predicted)
print('The y_test for comparison is : \n',y_final_test)

final_mse = mean_squared_error(y_final_test,y_final_predicted)

print('The corresponding MSE is : \n',final_mse)

The alpha selected, via 5-fold CV is : 
 0.005
The final y predicted over test set with the best selected alpha/lambda is : 
 [0.1860934  0.15148344 0.29906895 0.14071993 0.19790603 0.06532053
 0.20405817 0.36158261 0.10213257 0.31222131 0.22331815 0.42585507
 0.22175855 0.54636212 0.46796593 0.25990845 0.45993418 0.53356066
 0.16446817 0.42809459 0.0553898  0.07889475 0.22186409 0.19344426
 0.04457149 0.03214384 0.10298619 0.08479407 0.47080875 0.66035099
 0.15236782 0.17185734 0.21126994 0.21959573 0.11219579 0.12000663
 0.0454645  0.2564637  0.18134572 0.31686557 0.15820669 0.61840217
 0.28673562 0.37322367 0.34219628 0.36834998 0.35709491 0.27002495
 0.65204385 0.48139187 0.14817874 0.18135211 0.25117925 0.08226963
 0.11970327 0.16893702 0.16653793 0.1964538  0.243104   0.33798885
 0.14376483 0.16829817 0.23971508 0.1295967  0.14446404 0.18645027
 0.09426697 0.14808891 0.63251557 0.31444967 0.14442161 0.28966487
 0.31487716 0.15253236 0.47914683 0.23268045 0.11816086 0.10559953
 0.

In [17]:
coef_header = final_fit.coef_
coef_intercept = final_fit.intercept_
print(coef_header)
print(coef_intercept)

[ 0.         -0.          0.         -0.20859517  0.          0.
 -0.         -0.         -0.          0.          0.          0.02542096
 -0.         -0.         -0.         -0.          0.          0.
 -0.         -0.         -0.         -0.         -0.         -0.
  0.          0.         -0.          0.          0.          0.
  0.         -0.          0.         -0.         -0.         -0.
  0.         -0.          0.04194546  0.          0.          0.
  0.         -0.         -0.32241889 -0.         -0.         -0.
 -0.          0.          0.19617832  0.          0.          0.
  0.          0.          0.          0.          0.          0.
 -0.          0.          0.          0.          0.         -0.
  0.         -0.          0.01771124  0.         -0.          0.02838847
 -0.         -0.          0.01726978 -0.          0.          0.
  0.         -0.         -0.          0.         -0.         -0.
 -0.         -0.          0.          0.         -0.          0.
  0.     

As can be seen the coefficient vector for Lasso is extremely sparse, and quite a large number of features have been reduced to zero (made redundant)

In [19]:
feature_list = []
all_features = X.columns
i=0
for i in np.arange(0,122):
    if coef_header[i] != 0:
        feature_list.append(all_features[i])
        
print(feature_list)

['racePctWhite', 'pctUrban', 'MalePctDivorce', 'PctKids2Par', 'PctIlleg', 'PctPersDenseHous', 'HousVacant', 'PctVacantBoarded']


Thus these are the final selected features according to lasso